<a href="https://colab.research.google.com/github/Alok-16Rajput/Alok_llm_Assignment_task/blob/main/gradio_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [11]:
import gradio as gr
import pandas as pd

# Load default cleaned_data.csv for testing
default_file_path = "cleaned_data (1).csv"

# Define the analyze_logs function
def analyze_security_risks(file=None):
    """
    Analyze security risks in the uploaded dataset or the default dataset.
    """
    try:
        # Load the dataset
        if file is None:
            df = pd.read_csv(default_file_path)  # Use default file if none is provided
        else:
            df = pd.read_csv(file)

        # Check if the required columns exist
        required_columns = ['timestamp', 'user', 'action', 'source_IP', 'risk_score']
        if not all(col in df.columns for col in required_columns):
            return "The dataset must contain the following columns: " + ", ".join(required_columns)

        # High-risk IPs (threshold: risk_score > 50)
        high_risk_ips = df[df['risk_score'] > 0].groupby('source_IP').size().sort_values(ascending=False)
        high_risk_ips_count = len(high_risk_ips)
        high_risk_ips_output = high_risk_ips.reset_index(name='count').to_string(index=False)

        # Users with multiple failed logins (e.g., actions like 'FailedLogin')
        failed_login_users = df[df['action'].str.contains('FailedLogin', na=False)].groupby('user').size().sort_values(ascending=False)
        failed_login_users_count = len(failed_login_users)
        failed_login_users_output = failed_login_users.reset_index(name='count').to_string(index=False)

        # High-risk actions
        high_risk_actions = df[df['risk_score'] > 0].groupby('action').size().sort_values(ascending=False)
        high_risk_actions_count = len(high_risk_actions)
        high_risk_actions_output = high_risk_actions.reset_index(name='count').to_string(index=False)

        # Combine all analysis into a single string
        output = (
            f"### High-Risk IPs ({high_risk_ips_count} Total):\n" + high_risk_ips_output + "\n\n" +
            f"### Users with Multiple Failed Logins ({failed_login_users_count} Total):\n" + failed_login_users_output + "\n\n" +
            f"### High-Risk Actions ({high_risk_actions_count} Total):\n" + high_risk_actions_output
        )
        return output

    except Exception as e:
        return f"An error occurred: {e}"

# Build the Gradio app
with gr.Blocks() as demo:
    gr.Markdown("## Cloud Security Analyzer")
    gr.Markdown("Upload a log file (`cleaned_data.csv`) to analyze security risks.")

    file_input = gr.File(label="Upload Log File")
    output = gr.Textbox(label="Analysis Results", lines=15)

    # Add a button for analysis
    file_input.change(fn=analyze_security_risks, inputs=file_input, outputs=output)

# Launch the app
demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://55cfd9bafbd1b9bd82.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
